In [1]:
import sys
sys.path.append("..")

import torch.multiprocessing as mp
mp.set_start_method('spawn')
import torch
from torch.utils.data import DataLoader, Subset
from src.utils import project_root
from src.data.free_spoken_digits import Collator, Featurizer

In [2]:
import torchaudio
import torchaudio.functional as F

In [3]:
model_path = project_root()/"ckpt/free_spoken_digits.ckpt"

In [4]:


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

loaded_model = torch.load(model_path, map_location=torch.device("cpu")).to(device)
featurizer = Featurizer().to(device)

In [53]:

infer_audio_path = project_root()/"data/five.m4a"

infer_audio_path = project_root()/"data/one.m4a"
infer_audio_path

PosixPath('/home/user/grandeur/data/one.m4a')

In [37]:
!ls /home/user/grandeur/data/

CommonVoice  LJSpeech	  audio-dataset  free_spoken_digits  open_sst
Golos	     LibriSpeech  five.m4a	 one.m4a	     ruLibriSpeech


In [67]:


wav, sr = torchaudio.load(infer_audio_path)

resampled_wav = F.resample(wav, sr, featurizer.featurizer.sample_rate)

In [68]:
wav.shape

torch.Size([1, 37888])

In [69]:
resampled_wav.shape

torch.Size([1, 9472])

In [70]:
resampled_wav = resampled_wav.to(device)

In [71]:
length = resampled_wav.size(-1)

In [72]:
mel_inputs, length = featurizer(resampled_wav, length)

In [73]:
mel_inputs.shape

torch.Size([1, 64, 38])

In [74]:
length = torch.tensor([length]).long().to(device)

In [75]:
outputs = loaded_model(mel_inputs, length)

In [76]:
classes = outputs.argmax(-1)

In [79]:
for i, prob in enumerate(outputs[0].tolist()):
    print(i, prob)

0 0.33037957549095154
1 4.6266608238220215
2 0.9747194647789001
3 -2.496029853820801
4 5.11247444152832
5 -1.584223985671997
6 -3.6729774475097656
7 0.77483069896698
8 -1.7364847660064697
9 -1.0663233995437622


In [65]:
from IPython import display

In [66]:
display.display(display.Audio(data=resampled_wav.cpu().numpy(), rate=featurizer.featurizer.sample_rate))